In [1]:
import pyModeS as pms
import pandas as pd
import base64
from preprocess.decoder import Decoder

Lectura del fichero de despegues

In [2]:
datos  = pd.read_csv('test.csv', sep=';')
datos = datos.drop(columns=['Unnamed: 2'])
datos

,ts_kafka,message
0,1734667201534,oAANuYAVzS4ABLg3aB0=
1,1734667201534,kDQjSSgABjG4hW4fTbY=
2,1734667201534,XTxUM/+DTw==
3,1734667201534,kDQjShBRgxXDaCAWeDM=
4,1734667201534,jTRV1fgjAAIASbigrIs=
...,...,...
8354,1734667796655,IAAAALRFCw==
8355,1734667796655,IAAAALRFCw==
8356,1734667796655,jTlkQ5kItjeQCBj8Xms=
8357,1734667796655,IAAGM27mPA==


Los datos correspenden a la lectura que ha realizado la antena durante 1 minuto.

Son poco más de 8k filas, y la informacion de cada despeque se encuentra en la columna message, formateada en base 64.

Almacenaremos los datos en diccionarios.

In [3]:
hola = pms.tell(Decoder.base64toHex(datos.loc[0, "message"]))

                     Message: A0000DB98015CD2E0004B837681D 
                ICAO address: AA5F41 
             Downlink Format: 20 
                    Protocol: Mode-S Comm-B altitude reply 
                    Altitude: 21225 feet
                         BDS: BDS50 (Track and turn report) 
                  Roll angle: 0.0 degrees
                 Track angle: 130.4296875 degrees
                  Track rate: None degree/second
                Ground speed: 368 knots
               True airspeed: 368 knots


In [4]:
Decoder.processMessage(datos.loc[0, "message"], datos.loc[0,"ts_kafka"])

{'Timestamp (kafka)': 1734667201534,
 'Timestamp (date)': datetime.datetime(2024, 12, 20, 5, 0, 1, 534000),
 'Message (base64)': 'oAANuYAVzS4ABLg3aB0=',
 'Message (hex)': 'A0000DB98015CD2E0004B837681D',
 'ICAO': 'AA5F41',
 'Downlink Format': 20,
 'BDS': 'BDS50',
 'Roll angle (deg)': 0.0,
 'True track angle (deg)': 130.4296875,
 'Ground speed (kt)': 368,
 'Track angle rate (deg/sec)': None,
 'True airspeed (kt)': 368,
 'Altitude (ft)': 21225}

ModeS supports the decoding of following types of messages:

DF4 / DF20: Altitude code

DF5 / DF21: Identity code (squawk code)

DF17 / DF18: Automatic Dependent Surveillance-Broadcast (ADS-B)

TC=1-4 / BDS 0,8: Aircraft identification and category

TC=5-8 / BDS 0,6: Surface position

TC=9-18 / BDS 0,5: Airborne position

TC=19 / BDS 0,9: Airborne velocity

TC=28 / BDS 6,1: Airborne status [to be implemented]

TC=29 / BDS 6,2: Target state and status information [to be implemented]

TC=31 / BDS 6,5: Aircraft operational status [to be implemented]

DF20 / DF21: Mode-S Comm-B messages

BDS 1,0: Data link capability report

BDS 1,7: Common usage GICB capability report

BDS 2,0: Aircraft identification

BDS 3,0: ACAS active resolution advisory

BDS 4,0: Selected vertical intention

BDS 4,4: Meteorological routine air report (experimental)

BDS 4,5: Meteorological hazard report (experimental)

BDS 5,0: Track and turn report

BDS 6,0: Heading and speed report

In [3]:
df = datos.copy()
df_resultado = df.apply(lambda x: Decoder.processMessage(x["message"], x["ts_kafka"]), axis=1).apply(pd.Series)
df_resultado

,Timestamp (kafka),Timestamp (date),Message (base64),Message (hex),ICAO,Downlink Format,Flight status,BDS,Roll angle (deg),True track angle (deg),...,Wind shear level (0-3),Microburst level (0-3),Icing level (0-3),Wake vortex level (0-3),Static air temperature (C),Average static pressure (hPa),Radio height (ft),Overlay capability,Wind speed (kt) and direction (true) (deg),Humidity (%)
0,1734667201534,2024-12-20 05:00:01.534,oAANuYAVzS4ABLg3aB0=,A0000DB98015CD2E0004B837681D,AA5F41,20,NaN,BDS50,0.0,130.429688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1734667201534,2024-12-20 05:00:01.534,kDQjSSgABjG4hW4fTbY=,9034234928000631B8856E1F4DB6,342349,18,on-ground,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1734667201534,2024-12-20 05:00:01.534,XTxUM/+DTw==,5D3C5433FF834F,3C5433,11,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1734667201534,2024-12-20 05:00:01.534,kDQjShBRgxXDaCAWeDM=,9034234A10518315C36820167833,34234A,18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1734667201534,2024-12-20 05:00:01.534,jTRV1fgjAAIASbigrIs=,8D3455D5F82300020049B8A0AC8B,3455D5,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8354,1734667796655,2024-12-20 05:09:56.655,IAAAALRFCw==,20000000B4450B,342354,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8355,1734667796655,2024-12-20 05:09:56.655,IAAAALRFCw==,20000000B4450B,342354,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8356,1734667796655,2024-12-20 05:09:56.655,jTlkQ5kItjeQCBj8Xms=,8D3964439908B637900818FC5E6B,396443,17,on-ground,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8357,1734667796655,2024-12-20 05:09:56.655,IAAGM27mPA==,200006336EE63C,346419,4,on-ground,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from airstrip.airplane import Airplane

df_resultado['Timestamp (date)'] = pd.to_datetime(df_resultado['Timestamp (date)'])

airplanes = {}

# Iteramos sobre los mensajes del DataFrame
for _, row in df_resultado.iterrows():
    icao = row.get("ICAO")
    if not icao:
        continue
    if icao not in airplanes:
        airplanes[icao] = Airplane(icao)
    airplanes[icao].add_message(row)

# Obtenemos el timestamp final (por ahora asumiremos que es el máximo de todos los mensajes)
final_timestamp = df_resultado['Timestamp (date)'].max()



# Mostrar los resultados
for icao, airplane in airplanes.items():
    print(f"ICAO: {icao}")
    print(f"  Tiempo total en tierra: {airplane.total_ground_time} segundos")
    print(f"  Tiempo total en vuelo: {airplane.total_airborne_time} segundos")
    print("----")

ICAO: AA5F41
  Tiempo total en tierra: 451.01200000000006 segundos
  Tiempo total en vuelo: 131.70699999999997 segundos
----
ICAO: 342349
  Tiempo total en tierra: 0 segundos
  Tiempo total en vuelo: 0 segundos
----
ICAO: 3C5433
  Tiempo total en tierra: 581.0150000000002 segundos
  Tiempo total en vuelo: 13.395 segundos
----
ICAO: 34234A
  Tiempo total en tierra: 0 segundos
  Tiempo total en vuelo: 0 segundos
----
ICAO: 3455D5
  Tiempo total en tierra: 530.6240000000001 segundos
  Tiempo total en vuelo: 57.837 segundos
----
ICAO: 34234B
  Tiempo total en tierra: 0 segundos
  Tiempo total en vuelo: 0 segundos
----
ICAO: E06588
  Tiempo total en tierra: 309.443 segundos
  Tiempo total en vuelo: 245.97900000000004 segundos
----
ICAO: 346419
  Tiempo total en tierra: 547.015 segundos
  Tiempo total en vuelo: 48.106 segundos
----
ICAO: 4B18F5
  Tiempo total en tierra: 193.08899999999997 segundos
  Tiempo total en vuelo: 241.336 segundos
----
ICAO: ABD336
  Tiempo total en tierra: 219.74599